In [410]:
import os
import shutil
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import torch
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader, Subset
from torch.utils.tensorboard import SummaryWriter
from itertools import product
from torch.optim.lr_scheduler import ReduceLROnPlateau

random_state = 42

---
## Preparation

In [411]:
base_dir = "2_NN_outputs/"
model_dir = base_dir + "models/"
runs_dir = base_dir + "runs/"

# shutil.rmtree(model_dir, ignore_errors=True)
# shutil.rmtree(runs_dir, ignore_errors=True)
# os.makedirs(model_dir, exist_ok=True)
# os.makedirs(runs_dir, exist_ok=True)

### Device


In [412]:
# PyTorch Device
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print("Device: {}".format(device))

Device: cpu


### Set seeds


In [413]:
# For reproducibility, fix all the seeds
def fix_random(seed: int) -> None:
    """Fix all the possible sources of randomness.
    Args:
        seed: the seed to use.
    """
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True  # slower

### Data Layer


In [414]:
class MyDataset(Dataset):
    # Save X and y as Tensors, accordingly to the type of the data
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32, device=device)
        self.y = torch.tensor(y, dtype=torch.float32, device=device)

        self.num_features = X.shape[1]

    # Dataset size
    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx, :], self.y[idx]

### Neural Network Architecture

In [415]:
# Neural Network class
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, depth):
        super(NeuralNetwork, self).__init__()

        self.input_layer = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
        )

        self.hidden_layers = nn.Sequential(
            *[
                nn.Linear(hidden_size, hidden_size),
                nn.ReLU(),
            ]
            * (depth - 1)
        )

        self.output = nn.Linear(hidden_size, 1)

    def forward(self, x):
        h_in = self.input_layer(x)
        h_out = self.hidden_layers(h_in)
        out = self.output(h_out)
        return out

    def _get_description(self):
        return "Linear(all same size)+ReLU"

### Early Stopper

In [416]:
class EarlyStopper:
    def __init__(self, patience=1, delta=0):
        self.patience: int = patience
        self.delta: int = delta
        self.counter = 0
        self.min_validation_loss = float("inf")

    def early_stop(self, validation_loss):
        if validation_loss <= (self.min_validation_loss - self.delta):
            self.min_validation_loss = validation_loss
            self.counter = 0
            return False
        else:
            self.counter += 1
            # print("\tEarly stopper: {}/{} epochs".format(self.counter, self.patience))
            if self.counter >= self.patience:
                return True

### Train Function


In [417]:
# Function for the training process
def train_model(
    model: nn.Module,  # instance of class to train
    criterion,  # instance of loss function
    optimizer,  # instance of optimizer
    epochs,  # number of
    train_loader: DataLoader,
    val_loader: DataLoader,
    scheduler,
    device,  # to train on
    log_writer,
    log_name,
):
    n_iter = 0
    best_valid_loss = float("inf")  # initialized to worst possible value

    early_stopper = EarlyStopper(patience=10, delta=0.1)

    # EPOCHS
    for epoch in range(epochs):
        model.train()  # activate training mode (for BatchNorm or Dropout)

        # BATCHES
        for data, targets in train_loader:  # get_item from MyDataset class (single item or batch)
            data, targets = data.to(device), targets.to(device)

            optimizer.zero_grad()  # gradient to zero

            # Forward pass
            y_pred = model(data)

            # Compute Loss
            loss = criterion(y_pred.squeeze(), targets)  # reshape because MSELoss requires same dimensionality
            log_writer.add_scalar("Loss train/batches", loss, n_iter)  # plot the batches

            # Backward pass
            loss.backward()
            optimizer.step()

            n_iter += 1

        # Valuation
        y_test, y_pred = test_model(model, val_loader, device)
        loss_val = criterion(y_pred.squeeze(), y_test)
        log_writer.add_scalar("Loss val/epochs", loss_val, epoch)  # plot the epochs

        scheduler.step(loss_val)

        # Save the model with best loss through the epochs
        if loss_val.item() < best_valid_loss:
            best_valid_loss = loss_val.item()
            torch.save(model.state_dict(), model_dir + log_name)

        # Early Stopping
        if early_stopper.early_stop(loss_val.item()):
            print("Early stopped!")
            break

### Test Function


In [418]:
def test_model(model: nn.Module, data_loader: DataLoader, device) -> tuple[Tensor, Tensor, Tensor]:
    model.eval()  # activate evaluation mode (for BatchNorm or Dropout)

    y_pred = []
    y_test = []

    for data, targets in data_loader:
        data, targets = data.to(device), targets.to(device)

        y_pred.append(model(data))  # accumulate predictions
        y_test.append(targets)  # accumulate labels

    y_test = torch.stack(y_test).squeeze()
    y_pred = torch.stack(y_pred).squeeze()
    return y_test, y_pred

---

### Data Acquisition


In [419]:
df = pd.read_csv("train.csv").drop_duplicates()
print("Rows: ", df.shape[0])
print("Columns: ", df.shape[1])

df_X = df.iloc[:, 1:].values
df_y = df.iloc[:, 0].values
indices = np.arange(df_X.shape[0])

# Separate indices in train/val/set
train_idx, test_idx = train_test_split(indices, test_size=0.2, stratify=df_y, random_state=random_state)
train_idx, val_idx = train_test_split(train_idx, test_size=0.2, stratify=df_y[train_idx], random_state=random_state)

Rows:  252123
Columns:  91


### Preprocessing

In [420]:
# %%script false --no-raise-error
from sklearn import preprocessing
from sklearn.covariance import OAS
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.kernel_approximation import Nystroem
from sklearn.pipeline import Pipeline

# pipeline = Pipeline([("raw", None)])
# preprocess_name = "RAW"

# pipeline = Pipeline(steps=[("min-max", preprocessing.MinMaxScaler())])
# preprocess_name = "MinMax"

pipeline = Pipeline(
    steps=[
        ("std", preprocessing.StandardScaler()),
        ("l2", preprocessing.Normalizer(norm="l2")),
    ]
)
preprocess_name = "STD+L2"

# pipeline = Pipeline(
#     steps=[
#         ("min-max", preprocessing.MinMaxScaler()),
#         ("lmax", preprocessing.Normalizer(norm="max")),
#         ("lda", LinearDiscriminantAnalysis(solver="eigen", shrinkage=None, covariance_estimator=OAS())),
#     ]
# )
# preprocess_name = "MinMax+Lmax+LDA"

# pipeline = Pipeline(
#     [
#         ("min-max", preprocessing.MinMaxScaler()),
#         ("lda", LinearDiscriminantAnalysis(solver="eigen", covariance_estimator=OAS())),
#         ("nys", Nystroem(random_state=random_state, n_jobs=-1, gamma=0.010, n_components=1000)),
#     ]
# )
# preprocess_name = "MinMax+LDA+NYS"

pipeline.fit(df_X[train_idx], df_y[train_idx])

df_X_t = pipeline.transform(df_X)

### Hyperparameters


In [421]:
hidden_size = [128]  # 128, 256, 512, 1024]

batch_size = [32]  # [8, 16, 32, 64]

# dropout_p = [0.2, 0.3]  # 0.2

depth = [3]  # [3, 4, 5]

learning_rate = [0.0001]  # [0.001, 0.1]

num_epochs = [200]

# It is mathematically the preferred loss function under the inference framework of maximum likelihood if the distribution of the target variable is Gaussian.
criterion = [nn.MSELoss()]
## provare anche
# Mean Absolute Error (MAE) -> less sensitive to outliers
# Huber Loss -> combination of MAE
# Log-Cosh Loss -> smooth approximation of the MAE loss function. It is less sensitive to outliers than MSE and more robust than MAE.

optimizer = [torch.optim.RMSprop]  # Adam, SGD

scheduler_patience = [7]
scheduler_threshold = [1.0]


hyperparameters = list(
    product(
        hidden_size,
        depth,
        num_epochs,
        batch_size,
        learning_rate,
        criterion,
        optimizer,
        scheduler_patience,
        scheduler_threshold,
    )
)
n_comb = len(hyperparameters)
print(f"Combinations size: {n_comb}")

Combinations size: 1


### Run

In [422]:
best_loss = np.inf
best_combination = None
best_model = None

results_tt = pd.DataFrame(
    columns=[
        "R^2",
        "MSE",
        "preprocess",
        "NN_architecture",
        "depth",
        "hidden_size",
        "batch_size",
        "learning_rate",
        "num_epochs",
        "criterion",
        "optimizer",
        "scheduler_patience",
        "scheduler_threshold",
    ]
)

for n_iter, (
    hidden_size,
    depth,
    num_epochs,
    batch_size,
    learning_rate,
    criterion,
    optimizer,
    scheduler_patience,
    scheduler_threshold,
) in enumerate(hyperparameters):
    fix_random(random_state)
    print("\n===========================")
    print(f"Iteration {n_iter+1}/{n_comb}")

    # Data Loaders
    my_dataset = MyDataset(df_X_t, df_y)
    train_loader = DataLoader(Subset(my_dataset, train_idx), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(Subset(my_dataset, val_idx), batch_size=1)
    test_loader = DataLoader(Subset(my_dataset, test_idx), batch_size=1)

    model = NeuralNetwork(my_dataset.num_features, hidden_size, depth=depth)
    model.to(device)

    optimizer = optimizer(model.parameters(), lr=learning_rate, momentum=0.9)
    scheduler = ReduceLROnPlateau(
        optimizer, patience=scheduler_patience, threshold=scheduler_threshold, threshold_mode="abs", verbose=True
    )

    log_name = (
        preprocess_name
        + "--"
        + model._get_description()
        + "--"
        + f"depth={depth}--hidden_size={hidden_size}--batch_size={batch_size}--lr={learning_rate}"
        + "--"
        + f"sched_pat={scheduler_patience}--sched_thr={scheduler_threshold}"
        + "--"
        + f"opt={ optimizer.__class__.__name__}"
    )
    print(log_name)
    print(model)
    log_writer = SummaryWriter(runs_dir + log_name)

    train_model(
        model,
        criterion,
        optimizer,
        num_epochs,
        train_loader,
        val_loader,
        scheduler,
        device,
        log_writer,
        log_name,
    )

    # Load best model (saven in the train function)
    model.load_state_dict(torch.load(model_dir + log_name))
    model.to(device)

    # Test
    y_true, y_pred = test_model(model, test_loader, device)
    test_loss = criterion(y_pred, y_true)
    print(f"\nTest loss: {test_loss:.4f}")
    mse = mean_squared_error(y_true.detach().numpy(), y_pred.detach().numpy())
    r2 = r2_score(y_true.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
    print(f"R^2: {r2:.4f}, MSE:{mse:.4f}")

    results_tt.loc[len(results_tt)] = [
        r2,
        mse,
        preprocess_name,
        model._get_description(),
        depth,
        hidden_size,
        batch_size,
        learning_rate,
        num_epochs,
        criterion,
        optimizer.__class__.__name__,
        scheduler_patience,
        scheduler_threshold,
    ]
    results_tt.to_csv("2_output.csv", mode="a", index=False, header=False)

    if test_loss < best_loss:
        best_model = model
        best_combination = results_tt.loc[len(results_tt) - 1]

    log_writer.flush()
    log_writer.close()


Iteration 1/1
STD+L2--Linear(all same size)+ReLU--depth=3--hidden_size=128--batch_size=32--lr=0.0001--sched_pat=7--sched_thr=1.0--opt=RMSprop
NeuralNetwork(
  (input_layer): Sequential(
    (0): Linear(in_features=90, out_features=128, bias=True)
    (1): ReLU()
  )
  (hidden_layers): Sequential(
    (0): Linear(in_features=128, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
  )
  (output): Linear(in_features=128, out_features=1, bias=True)
)
Epoch 00014: reducing learning rate of group 0 to 1.0000e-05.


### Best results

In [ ]:
print("Best combination:")
print(best_combination)

Best combination:
R^2                                      0.322598
MSE                                      74.63176
preprocess                                 STD+L2
NN_architecture        Linear(all same size)+ReLU
depth                                           4
hidden_size                                   256
batch_size                                     32
learning_rate                              0.0001
num_epochs                                    200
criterion                               MSELoss()
optimizer                                 RMSprop
scheduler_patience                              7
scheduler_threshold                           1.0
Name: 3, dtype: object


In [ ]:
pd.read_csv("2_output.csv").sort_values(by="R^2", ascending=False).style.format(precision=4)

,R^2,MSE,preprocess,NN_architecture,depth,hidden_size,batch_size,learning_rate,num_epochs,criterion,optimizer,scheduler_patience,scheduler_threshold
18,0.3453,72.1261,STD+L2,Linear(all same size)+ReLU,3,256,32,0.0001,200,MSELoss(),RMSprop,7.0000,1.0000
21,0.3406,72.6518,STD+L2,Linear(all same size)+ReLU,4,128,32,0.0001,200,MSELoss(),RMSprop,7.0000,1.0000
23,0.3406,72.6518,STD+L2,Linear(all same size)+ReLU,4,128,32,0.0001,200,MSELoss(),RMSprop,7.0000,1.0000
26,0.3406,72.6518,STD+L2,Linear(all same size)+ReLU,4,128,32,0.0001,200,MSELoss(),RMSprop,7.0000,1.0000
14,0.3312,73.6791,STD+Lmax,Linear(all same size)+ReLU,3,256,32,0.0001,200,MSELoss(),RMSprop,7.0000,1.0000
17,0.3305,73.7619,STD+L1,Linear(all same size)+ReLU,3,256,32,0.0001,200,MSELoss(),RMSprop,7.0000,1.0000
24,0.3299,73.8325,STD+L2,Linear(all same size)+ReLU,2,256,32,0.0001,200,MSELoss(),RMSprop,7.0000,1.0000
27,0.3299,73.8325,STD+L2,Linear(all same size)+ReLU,2,256,32,0.0001,200,MSELoss(),RMSprop,7.0000,1.0000
16,0.3297,73.8459,STD+Lmax+LDA+NYS,Linear(all same size)+ReLU,3,256,32,0.0001,200,MSELoss(),RMSprop,7.0000,1.0000
15,0.3286,73.9717,STD+Lmax+LDA,Linear(all same size)+ReLU,3,256,32,0.0001,200,MSELoss(),RMSprop,7.0000,1.0000


In [ ]:
y_true, y_pred = test_model(best_model, test_loader, device)

mse = mean_squared_error(y_true.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
r2 = r2_score(y_true.cpu().detach().numpy(), y_pred.cpu().detach().numpy())

print("MSE:", mse)
print("R2:", r2)

MSE: 74.63176
R2: 0.3225983965233421
